In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import urllib.request, json
from shapely.geometry import Polygon, MultiPolygon, LineString, Point
from os.path import join
from fiona.crs import from_epsg
import matplotlib.pyplot as plt
from os.path import join
from fiona.crs import from_epsg
from shapely.geometry import Point, LineString, Polygon, mapping
from descartes import PolygonPatch
import fiona
import osmnx as ox
import networkx as nx
import urllib.request, json

In [2]:
path_parks= r'M:\NY4P Metrics Research 18034\03_GIS+ANALYSIS\01_DATA\RAW\Parks Historical Permits'

# Load Data
* perms: permit data
* parks: parks shapefile
* nhoods: bayst, lic, and bushwick
* walksheds: shapefile of all walksheds (length=818)
* cds: shapefile of all community districts, useful as a proxy for neighborhoods

In [3]:
perms= pd.read_csv(join(path_parks, '190618_ParksPermits_ADDCOLS.csv'), low_memory=False, index_col=0)

In [4]:
parks= gpd.read_file(r"M:\NY4P Metrics Research 18034\03_GIS+ANALYSIS\01_DATA\RAW\NYC_PARKS_DATA\NYCPARKS_ALLSITES.shp")

In [5]:
bayst= gpd.read_file(r"M:\NY4P Metrics Research 18034\03_GIS+ANALYSIS\01_DATA\Walk-to-a-Park Summer 2019 Data\Walk-to-a-Park Summer 2019 Data\Walk-to-a-Park_BayStreetCorridorAccessPoints.shp")

In [6]:
lic1= gpd.read_file(r"M:\NY4P Metrics Research 18034\03_GIS+ANALYSIS\01_DATA\Walk-to-a-Park Summer 2019 Data\Walk-to-a-Park Summer 2019 Data\Walk-to-a-Park_LIC_Parks1acreorlessAccessPoints.shp").set_index('gispropnum')
lic2= gpd.read_file(r"M:\NY4P Metrics Research 18034\03_GIS+ANALYSIS\01_DATA\Walk-to-a-Park Summer 2019 Data\Walk-to-a-Park Summer 2019 Data\Walk-to-a-Park_LIC_Parks20acresorgreater.shp").set_index('gispropnum')
lic3= gpd.read_file(r"M:\NY4P Metrics Research 18034\03_GIS+ANALYSIS\01_DATA\Walk-to-a-Park Summer 2019 Data\Walk-to-a-Park Summer 2019 Data\Walk-to-a-Park_LIC_Parksbtwn1and20acresAccessPoints.shp").set_index('gispropnum')

In [7]:
lic= pd.concat([lic1,lic2,lic3],axis=0,sort=True).reset_index()

In [8]:
bushwick= gpd.read_file(r"M:\NY4P Metrics Research 18034\03_GIS+ANALYSIS\01_DATA\Walk-to-a-Park Summer 2019 Data\Walk-to-a-Park Summer 2019 Data\Walk-to-a-Park_BushwickAccessPoints.shp")

In [9]:
bayst_parks= bayst['gispropnum'].unique().tolist()
lic_parks= lic['gispropnum'].unique().tolist()
bushwick_parks= bushwick['gispropnum'].unique().tolist()

In [10]:
parkentrances= gpd.read_file(r"M:\NY4P Metrics Research 18034\03_GIS+ANALYSIS\01_DATA\Walk-to-a-Park Summer 2019 Data\Walk-to-a-Park Summer 2019 Data\Walk to a Park 2019.shp")

In [15]:
walksheds= gpd.read_file(r"M:\NY4P Metrics Research 18034\03_GIS+ANALYSIS\01_DATA\Processed\Isochrones\190718_Isochrones_JG\190806_allparks10minwalkshed.shp")

In [16]:
walksheds= walksheds.set_index('GISPROPNUM')

In [19]:
cds= gpd.read_file(r"S:\NYC_GIS_DATA\community districts\nycd.shp")

In [22]:
uwswalk= gpd.sjoin(walksheds,cds[cds['BoroCD']==107],op='intersects',how='inner')

In [42]:
parkslope= gpd.sjoin(walksheds,cds[cds['BoroCD']==306],op='intersects',how='inner')

In [43]:
evwalk= gpd.sjoin(walksheds,cds[cds['BoroCD']==103],op='intersects',how='inner')

### Helpers
* Park filter returns a new dataframe filtered for the GISPROPNUMs specified by park_sel
* Score_Filter does the same thing, but for numeric scores not grades.

In [29]:
def Park_Filter(df,park_sel):
    '''
    Returns a new dataframe filtered for the selection of neighborhood parks.
    Works with letter grades only.
    '''
    #df= pd.merge(df,parks[['GISPROPNUM','SIGNNAME']], on='GISPROPNUM', how='inner')
    t= df[df['GISPROPNUM'].isin(park_sel)].set_index('GISPROPNUM')[['TYPECATEGO','SIGNNAME','tp_grade','tpa_grade','tppop_grade','tppopdens_grade']]
    return t

In [30]:
def Score_Filter(df,park_sel):
    '''
    Returns a new dataframe of the df above filtered for the selection of neighborhood parks (park_sel).
    It only works for numeric scores, not for letter grades.
    '''
    df= pd.merge(df,parks[['GISPROPNUM','SIGNNAME']], on='GISPROPNUM', how='inner')
    df= pd.merge(df.copy(), NY4Pscores2.drop('TYPECATEGO',axis=1),  on='GISPROPNUM', how='inner' )
    t= df[df['GISPROPNUM'].isin(park_sel)].set_index('GISPROPNUM')[['TYPECATEGO','SIGNNAME','tp','tpa','tppop','tppopdens','tp_palmm','tpa_palmm','tppop_palmm','tppopdens_palmm']]
    return t

## Score Comparison
* NY4Pscores - with normalization
* NY4Pscores2 - no normalization, just the values

In [34]:
score_path= r"M:\NY4P Metrics Research 18034\03_GIS+ANALYSIS\01_DATA\Processed\Initial Options - Scores"

In [35]:
NY4Pscores= pd.read_csv(join(score_path,"190729_TParks by Acre and Pop.csv"))

In [36]:
NY4Pscores2= pd.read_csv(join(score_path,"190729_TParks by Acre and Pop_NoNormalization.csv" ))

In [38]:
DPRscores= pd.read_csv(join(score_path,"190729_TParks by Acre and Pop_DPR.csv"))

In [39]:
alt1scores= pd.read_csv(join(score_path,"190729_TParks by Acre and Pop_Alt1.csv"))

In [44]:
evparks= evwalk.index.unique()

In [45]:
uws_parks= uwswalk.index.unique()

In [50]:
parkslope_parks= parkslope.index.unique()

In [51]:
evscores= Score_Filter(NY4Pscores, evparks)
evscores['Neighborhood']= 'Lower East Side'
evscores;

In [52]:
licscores= Score_Filter(NY4Pscores, lic_parks)
licscores['Neighborhood']= 'Long Island City'

In [53]:
uwsscores= Score_Filter(NY4Pscores, uws_parks)
uwsscores['Neighborhood']= 'Upper West Side'

In [54]:
baystscores= Score_Filter(NY4Pscores, bayst_parks)
baystscores['Neighborhood']= 'Bay St. Corridor'
bushwickscores= Score_Filter(NY4Pscores, bushwick_parks)
bushwickscores['Neighborhood']= 'Bushwick'
parkslopescores= Score_Filter(NY4Pscores, parkslope_parks)
parkslopescores['Neighborhood']= 'Park Slope'

In [55]:
nhoodscores= pd.concat([licscores, baystscores, bushwickscores, evscores, uwsscores, parkslopescores],axis=0)

In [56]:
nhoodscores.to_csv(r"M:\NY4P Metrics Research 18034\03_GIS+ANALYSIS\01_DATA\Processed\Initial Options - Scores\Nhood_scores.csv")

In [57]:
lc= ['sum']+['mean' for x in ['tpa','tppop','tppopdens','tp_palmm','tpa_palmm','tppop_palmm','tppopdens_palmm']]

In [58]:
lc[2]= 'sum'
lc

['sum', 'mean', 'sum', 'mean', 'mean', 'mean', 'mean', 'mean']

In [59]:
nhoodsum= nhoodscores.groupby('Neighborhood').agg(dict(zip(['tp','tpa','tppop','tppopdens','tp_palmm','tpa_palmm','tppop_palmm','tppopdens_palmm'], lc)))

In [61]:
nhoodsum
#Summary table by study area and comparison neihgborhoods: of both scores and original values

,tp,tpa,tppop,tppopdens,tp_palmm,tpa_palmm,tppop_palmm,tppopdens_palmm
Neighborhood,,,,,,,,
Bay St. Corridor,644.0,26.369016,0.039024,3.443430,37.000000,45.250000,44.000000,52.125000
Bushwick,1989.0,22.000576,0.432665,1.909449,34.666667,49.083333,39.041667,39.500000
Long Island City,1535.0,28.136769,0.208426,1.676015,39.952381,48.000000,52.095238,50.285714
Lower East Side,9454.0,93.930468,0.203869,2.052887,48.705882,48.000000,46.176471,48.235294
Park Slope,13225.0,61.596660,0.323477,8.101612,47.440000,54.880000,51.640000,53.360000
Upper West Side,46988.0,76.892271,0.257145,26.426437,52.937500,53.437500,48.250000,50.875000


In [63]:
nhoodsum.to_csv(r"M:\NY4P Metrics Research 18034\03_GIS+ANALYSIS\01_DATA\Processed\Initial Options - Scores\190806_Nhood_summary.csv")

# DPR Categories
* filtering for scores by neighborhood:
    - east village
    - bay st
    - bushwick
    - lic
* uses DPR categories (not NY4P categories)
* letter grades, not numeric grades

In [114]:
parksacre= pd.read_csv(r"M:\NY4P Metrics Research 18034\03_GIS+ANALYSIS\01_DATA\Processed\Initial Options\190618_TParks by Acre.csv",index_col=0).set_index('GISPROPNUM')

In [118]:
grades= gpd.read_file(r"M:\NY4P Metrics Research 18034\03_GIS+ANALYSIS\01_DATA\Processed\Initial Options\190719_NewGrades_2.shp")

In [119]:
parkacres= parksacre['tpa_grade'].dropna()

In [120]:
grades= pd.merge(parkacres.reset_index(),grades.copy(),on='GISPROPNUM',how='right').sort_index(1)

In [122]:
grades.shape

(531, 32)

In [123]:
grades.rename(columns={'tppop_grad':'tppop_grade','tppopden_2':'tppopdens_grade','index':'GISPROPNUM'},inplace=True)

In [124]:
grades.head().loc[:,['tp_grade','tpa_grade','tppop_grade','tppopdens_grade']]

,tp_grade,tpa_grade,tppop_grade,tppopdens_grade
0,C-,D,B-,C+
1,F,D+,D,F
2,C+,C-,C+,B-
3,D+,F,C-,C+
4,F,D-,F,D-


In [125]:
evgrades= Park_Filter(grades,evparks)

In [126]:
baystgrades= Park_Filter(grades,bayst_parks)

In [127]:
bushwickgrades= Park_Filter(grades,bushwick_parks)

In [128]:
licgrades= Park_Filter(grades,lic_parks)

In [129]:
uwsgrades= Park_Filter(grades,uws_parks)

In [130]:
parkslopegrades= Park_Filter(grades,parkslope_parks)

# NY4P Park Categories

In [131]:
newgrades= gpd.read_file(r"M:\NY4P Metrics Research 18034\03_GIS+ANALYSIS\01_DATA\Processed\Initial Options - New Categories\190729_TParks by Acre and Pop.csv",index_col=0)

In [132]:
newgrades= pd.merge(newgrades,parks[['GISPROPNUM','SIGNNAME']],on='GISPROPNUM',how='left')

In [133]:
evnewgrades= Park_Filter(newgrades,evparks)

In [134]:
baystnewgrades= Park_Filter(newgrades,bayst_parks)

In [135]:
bushwicknewgrades= Park_Filter(newgrades,bushwick_parks)

In [136]:
licnewgrades= Park_Filter(newgrades,lic_parks)

In [137]:
uwsnewgrades= Park_Filter(newgrades,uws_parks)

In [138]:
parkslopenewgrades= Park_Filter(newgrades,parkslope_parks)

In [139]:
newgrades.columns

Index(['GISPROPNUM', 'TYPECATEGO', 'tp_grade', 'tpa_grade', 'tppop_grade',
       'tppopdens_grade', 'geometry', 'SIGNNAME'],
      dtype='object')

## Comparison of new and old grades by park type
* What park types consistently score Fs?
* What park types change?
* grades= DPR categories
* newgrades= NY4P categories

In [152]:
pd.merge(grades, parks, on='GISPROPNUM').groupby('TYPECATEGO_x')['tp_grade'].value_counts().sort_values(ascending=False).head(10)

TYPECATEGO_x                 tp_grade
Neighborhood Park            F           136
Playground                   F           117
Jointly Operated Playground  F            84
Community Park               F            26
Neighborhood Park            D            10
Flagship Park                F            10
Jointly Operated Playground  D             9
Community Park               D             8
                             B-            7
Neighborhood Park            C             7
Name: tp_grade, dtype: int64

In [153]:
pd.merge(newgrades, parks, on='GISPROPNUM').groupby('TYPECATEGO_x')['tp_grade'].value_counts().sort_values(ascending=False).head(10)

TYPECATEGO_x  tp_grade
NP            F           287
PP            F           169
LP            F            30
NP            D            11
              D-           10
              C-            9
LP            D             8
              D+            8
NP            C             8
PP            D+            6
Name: tp_grade, dtype: int64

In [154]:
pd.merge(grades, parks, on='GISPROPNUM').groupby('TYPECATEGO_y')['tpa_grade'].value_counts().sort_values(ascending=False).head(10)

TYPECATEGO_y                 tpa_grade
Jointly Operated Playground  F            99
Playground                   F            86
Neighborhood Park            F            75
Community Park               F            31
Neighborhood Park            C            20
                             C-           16
Playground                   D            14
                             D-           11
Neighborhood Park            D-           11
                             B-           10
Name: tpa_grade, dtype: int64

In [155]:
pd.merge(newgrades, parks, on='GISPROPNUM').groupby('TYPECATEGO_x')['tpa_grade'].value_counts().sort_values(ascending=False).head(10)

TYPECATEGO_x  tpa_grade
NP            F            203
PP            F            197
NP            D             28
              D+            23
              C-            22
              C             20
              D-            11
              C+            11
LP            F              9
              C              8
Name: tpa_grade, dtype: int64

In [156]:
pd.merge(grades, parks, on='GISPROPNUM').groupby('TYPECATEGO_y')['tppop_grade'].value_counts().sort_values(ascending=False).head(10)

TYPECATEGO_y                 tppop_grade
Neighborhood Park            F              141
Playground                   F              105
Jointly Operated Playground  F               95
Community Park               F               30
Neighborhood Park            D               16
Community Park               C-               9
                             C                8
Jointly Operated Playground  D                8
Playground                   D                6
Neighborhood Park            D+               6
Name: tppop_grade, dtype: int64

In [157]:
pd.merge(newgrades, parks, on='GISPROPNUM').groupby('TYPECATEGO_x')['tppop_grade'].value_counts().sort_values(ascending=False).head(10)

TYPECATEGO_x  tppop_grade
NP            F              287
PP            F              155
LP            F               22
NP            D               18
              D+              13
LP            C-              11
NP            D-               9
PP            C+               8
LP            D                7
PP            D                6
Name: tppop_grade, dtype: int64

In [158]:
pd.merge(grades, parks, on='GISPROPNUM').groupby('TYPECATEGO_y')['tppopdens_grade'].value_counts().sort_values(ascending=False).head(10)

TYPECATEGO_y                 tppopdens_grade
Neighborhood Park            F                  147
Playground                   F                  109
Jointly Operated Playground  F                   93
Community Park               F                   23
Neighborhood Park            D-                  10
Jointly Operated Playground  D                   10
Playground                   D                    8
Community Park               C+                   7
Flagship Park                F                    7
Community Park               B-                   6
Name: tppopdens_grade, dtype: int64

In [159]:
pd.merge(newgrades, parks, on='GISPROPNUM').groupby('TYPECATEGO_x')['tppopdens_grade'].value_counts().sort_values(ascending=False).head(10)

TYPECATEGO_x  tppopdens_grade
NP            F                  286
PP            F                  152
LP            F                   22
NP            D                   15
              D+                  14
              D-                  11
LP            D                   10
PP            D                    6
              C-                   6
LP            C                    6
Name: tppopdens_grade, dtype: int64

# No Outliers
* NY4P Categories and No Outliers
* Outliers are flagship parks using DPR's categorizations as well as Bryant Park

In [160]:
nooutliers= pd.read_csv(r"M:\NY4P Metrics Research 18034\03_GIS+ANALYSIS\01_DATA\Processed\Initial Options - New Categories\190729_TParks by Acre and Pop_NoOutliers.csv",index_col=0)

In [161]:
nooutliers= pd.merge(nooutliers,parks[['GISPROPNUM','SIGNNAME']],on='GISPROPNUM',how='left')

In [162]:
evnooutliers= Park_Filter(nooutliers,evparks)

In [163]:
baystnooutliers= Park_Filter(nooutliers,bayst_parks)

In [164]:
bushwicknooutliers= Park_Filter(nooutliers,bushwick_parks)

In [165]:
licnooutliers= Park_Filter(nooutliers,lic_parks)

In [166]:
uwsnooutliers= Park_Filter(nooutliers,uws_parks)

In [167]:
parkslopenooutliers= Park_Filter(nooutliers,parkslope_parks)

# No Outliers 2
* Outliers are removed if total permitted events > 95th percentile

In [168]:
nooutliers2= pd.read_csv(r"M:\NY4P Metrics Research 18034\03_GIS+ANALYSIS\01_DATA\Processed\Initial Options - New Categories\190729_TParks by Acre and Pop_NoOutliers2.csv")

In [169]:
nooutliers2= pd.merge(nooutliers2, parks[['GISPROPNUM','SIGNNAME']],on='GISPROPNUM',how='left')

In [170]:
evnooutliers2= Park_Filter(nooutliers2,evparks)

In [171]:
uwsnooutliers2= Park_Filter(nooutliers2,uws_parks)

In [172]:
parkslopenooutliers2= Park_Filter(nooutliers2,parkslope_parks)

In [173]:
baystnooutliers2= Park_Filter(nooutliers2,bayst_parks)

In [174]:
bushwicknooutliers2= Park_Filter(nooutliers2,bushwick_parks)

In [175]:
licnooutliers2= Park_Filter(nooutliers2,lic_parks)

# Alternative 1
* Topcodes outlier parks
* Outliers are parks with total permitted events > 95th percentile within each NY4P Park Type

In [176]:
alt1= pd.read_csv(r"M:\NY4P Metrics Research 18034\03_GIS+ANALYSIS\01_DATA\Processed\Initial Options - New Categories\190729_TParks by Acre and Pop_Alt1.csv")

In [177]:
alt1= pd.merge(alt1, parks[['GISPROPNUM','SIGNNAME']],on='GISPROPNUM',how='left')

In [178]:
evalt= Park_Filter(alt1,evparks)

In [179]:
baystalt= Park_Filter(alt1,bayst_parks)

In [180]:
bushwickalt= Park_Filter(alt1,bushwick_parks)

In [181]:
licalt= Park_Filter(alt1,lic_parks)

In [182]:
parkslopealt= Park_Filter(alt1,parkslope_parks)

In [183]:
uwsalt= Park_Filter(alt1,uws_parks)

# Comparison of Grades
* compare_grades: a function that prints a comparison, tallying up the number of parks with better grades.

In [193]:
def compare_grades(df,df1,df2,grade_type):
    '''
    Compares grades across the three datasets: DPR categories, NY4P categories, and new categories with no outliers.
    Prints the number of parks with higher scores.
    Grade_type is either: tp_grade, tpa_grade, tppop_grade, tppopdens_grade.
    '''
    inters= df.index.intersection(df1.index).intersection(df2.index)
    df= df.loc[inters]
    df1= df1.loc[inters]
    df2= df2.loc[inters]
    grades= ['A','B','C','D','F']
    for i in range(len(grades)):
        df.loc[(df[grade_type].str.contains(grades[i]))&(df[grade_type].notna()), grade_type]= 100-(i*10)
        df1.loc[df1[grade_type].str.contains(grades[i])&(df1[grade_type].notna()), grade_type]= 100-(i*10)
        df2.loc[df2[grade_type].str.contains(grades[i])&(df2[grade_type].notna()), grade_type]= 100-(i*10)
    
    comp= ((df[grade_type])<(df1[grade_type])).sum()
    comp1= ((df1[grade_type])<(df2[grade_type])).sum()
    comp2= ((df2[grade_type])<(df[grade_type])).sum()
    print('Total Number of Parks: {}'.format(len(inters)))
    print('')
    print('Comparisons for {}:'.format(grade_type))
    print('Comparing NY4P Categories to DPR Categories, the number of parks with better grades: {}'.format(comp))
    print('Comparing NY4P Categories with no outliers to NY4P Categories with Outliers, the number of parks with better grades: {}'.format(comp1))
    print('Comparing DPR Categories to NY4P Categories with no outliers, the number of parks with better grades: {}'.format(comp2))
    print('')
    return [comp, comp1, comp2]

In [194]:
# For the EV
for i in evgrades.drop(['SIGNNAME','TYPECATEGO'],axis=1).columns:
    compare_grades(evgrades,evnewgrades,evnooutliers,i)

Total Number of Parks: 30

Comparisons for tp_grade:
Comparing NY4P Categories to DPR Categories, the number of parks with better grades: 1
Comparing NY4P Categories with no outliers to NY4P Categories with Outliers, the number of parks with better grades: 2
Comparing DPR Categories to NY4P Categories with no outliers, the number of parks with better grades: 7

Total Number of Parks: 30

Comparisons for tpa_grade:
Comparing NY4P Categories to DPR Categories, the number of parks with better grades: 0
Comparing NY4P Categories with no outliers to NY4P Categories with Outliers, the number of parks with better grades: 0
Comparing DPR Categories to NY4P Categories with no outliers, the number of parks with better grades: 12

Total Number of Parks: 30

Comparisons for tppop_grade:
Comparing NY4P Categories to DPR Categories, the number of parks with better grades: 5
Comparing NY4P Categories with no outliers to NY4P Categories with Outliers, the number of parks with better grades: 4
Comparin

In [195]:
# For Bay St
for i in baystnewgrades.drop(['SIGNNAME','TYPECATEGO'],axis=1).columns:
    compare_grades(baystnooutliers2,baystnewgrades,baystnooutliers,i)

Total Number of Parks: 8

Comparisons for tp_grade:
Comparing NY4P Categories to DPR Categories, the number of parks with better grades: 0
Comparing NY4P Categories with no outliers to NY4P Categories with Outliers, the number of parks with better grades: 1
Comparing DPR Categories to NY4P Categories with no outliers, the number of parks with better grades: 3

Total Number of Parks: 8

Comparisons for tpa_grade:
Comparing NY4P Categories to DPR Categories, the number of parks with better grades: 0
Comparing NY4P Categories with no outliers to NY4P Categories with Outliers, the number of parks with better grades: 0
Comparing DPR Categories to NY4P Categories with no outliers, the number of parks with better grades: 1

Total Number of Parks: 8

Comparisons for tppop_grade:
Comparing NY4P Categories to DPR Categories, the number of parks with better grades: 0
Comparing NY4P Categories with no outliers to NY4P Categories with Outliers, the number of parks with better grades: 1
Comparing DP

In [196]:
# For Bushwick
for i in bushwickgrades.drop(['SIGNNAME','TYPECATEGO'],axis=1).columns:
    compare_grades(bushwickgrades,bushwicknewgrades,bushwicknooutliers,i)

Total Number of Parks: 24

Comparisons for tp_grade:
Comparing NY4P Categories to DPR Categories, the number of parks with better grades: 0
Comparing NY4P Categories with no outliers to NY4P Categories with Outliers, the number of parks with better grades: 1
Comparing DPR Categories to NY4P Categories with no outliers, the number of parks with better grades: 3

Total Number of Parks: 24

Comparisons for tpa_grade:
Comparing NY4P Categories to DPR Categories, the number of parks with better grades: 2
Comparing NY4P Categories with no outliers to NY4P Categories with Outliers, the number of parks with better grades: 0
Comparing DPR Categories to NY4P Categories with no outliers, the number of parks with better grades: 4

Total Number of Parks: 24

Comparisons for tppop_grade:
Comparing NY4P Categories to DPR Categories, the number of parks with better grades: 0
Comparing NY4P Categories with no outliers to NY4P Categories with Outliers, the number of parks with better grades: 1
Comparing

In [197]:
# For LIC
for i in licgrades.drop(['SIGNNAME','TYPECATEGO'],axis=1).columns:
    compare_grades(licgrades,licnewgrades,licnooutliers,i)

Total Number of Parks: 20

Comparisons for tp_grade:
Comparing NY4P Categories to DPR Categories, the number of parks with better grades: 0
Comparing NY4P Categories with no outliers to NY4P Categories with Outliers, the number of parks with better grades: 3
Comparing DPR Categories to NY4P Categories with no outliers, the number of parks with better grades: 4

Total Number of Parks: 20

Comparisons for tpa_grade:
Comparing NY4P Categories to DPR Categories, the number of parks with better grades: 4
Comparing NY4P Categories with no outliers to NY4P Categories with Outliers, the number of parks with better grades: 0
Comparing DPR Categories to NY4P Categories with no outliers, the number of parks with better grades: 5

Total Number of Parks: 20

Comparisons for tppop_grade:
Comparing NY4P Categories to DPR Categories, the number of parks with better grades: 2
Comparing NY4P Categories with no outliers to NY4P Categories with Outliers, the number of parks with better grades: 2
Comparing

In [198]:
# For UWS
for i in uwsgrades.drop(['SIGNNAME','TYPECATEGO'],axis=1).columns:
    compare_grades(uwsgrades,uwsnewgrades,uwsnooutliers,i)

Total Number of Parks: 15

Comparisons for tp_grade:
Comparing NY4P Categories to DPR Categories, the number of parks with better grades: 0
Comparing NY4P Categories with no outliers to NY4P Categories with Outliers, the number of parks with better grades: 4
Comparing DPR Categories to NY4P Categories with no outliers, the number of parks with better grades: 5

Total Number of Parks: 15

Comparisons for tpa_grade:
Comparing NY4P Categories to DPR Categories, the number of parks with better grades: 2
Comparing NY4P Categories with no outliers to NY4P Categories with Outliers, the number of parks with better grades: 0
Comparing DPR Categories to NY4P Categories with no outliers, the number of parks with better grades: 3

Total Number of Parks: 15

Comparisons for tppop_grade:
Comparing NY4P Categories to DPR Categories, the number of parks with better grades: 2
Comparing NY4P Categories with no outliers to NY4P Categories with Outliers, the number of parks with better grades: 0
Comparing

In [199]:
# For Park Slope
for i in uwsgrades.drop(['SIGNNAME','TYPECATEGO'],axis=1).columns:
    compare_grades(parkslopegrades,parkslopenewgrades,parkslopenooutliers,i)

Total Number of Parks: 20

Comparisons for tp_grade:
Comparing NY4P Categories to DPR Categories, the number of parks with better grades: 1
Comparing NY4P Categories with no outliers to NY4P Categories with Outliers, the number of parks with better grades: 2
Comparing DPR Categories to NY4P Categories with no outliers, the number of parks with better grades: 2

Total Number of Parks: 20

Comparisons for tpa_grade:
Comparing NY4P Categories to DPR Categories, the number of parks with better grades: 1
Comparing NY4P Categories with no outliers to NY4P Categories with Outliers, the number of parks with better grades: 0
Comparing DPR Categories to NY4P Categories with no outliers, the number of parks with better grades: 7

Total Number of Parks: 20

Comparisons for tppop_grade:
Comparing NY4P Categories to DPR Categories, the number of parks with better grades: 2
Comparing NY4P Categories with no outliers to NY4P Categories with Outliers, the number of parks with better grades: 3
Comparing

# Neighborhood Averages

In [200]:
def round_5(col):
    '''
    Rounds to the nearest multiple of 5.
    Takes in a series or a number.
    '''
    col= round(col/5)*5
    return col

In [202]:
def average_score(df1):
    '''
    Reverse engineers an average score using the letter grades (slightly inflates the actual grade).
    Prints the neighborhood average and the average by grade type.
    '''
    df= df1.copy().dropna()
    df['Average Score']= 0
    grades= ['A','A-','B+','B','B-','C+','C','C-','D+','D','D-','F']
    scores= list(range(100,40,-5))
    
    grade_type= ['tp_grade','tpa_grade','tppop_grade','tppopdens_grade']

    for j in grade_type:
        for i in range(len(grades)):
            df.loc[(df[j].str.contains(grades[i]))&(df[j].notna()), j]= 100-(i*5)
        df[j]= round_5(df[j])
        tavg= [grades[i] for i in range(len(scores)) if scores[i]==round_5(df[j].mean())][-1]
        print('The neighborhood average for {} is {}'.format(j,tavg))
        
    df['Average Score']= round_5(df[grade_type].mean(axis=1)) 
    
    avg= round_5( (df['Average Score'].sum() + 100 )/(len(df['Average Score'])+1) )
    avg_grade= [grades[i] for i in range(len(scores)) if scores[i]==avg]

    for m in range(len(scores)):
        df.loc[df['Average Score']==scores[m],'Average Score']= grades[m]
    
    print('')
    print('The Neighborhood Average is: {}'.format(avg_grade[-1]))
    return df['Average Score']

## East Village
* average numeric scores and 'average' letter scores

In [215]:
evnewgrades.sort_values('SIGNNAME')

,TYPECATEGO,SIGNNAME,tp_grade,tpa_grade,tppop_grade,tppopdens_grade
GISPROPNUM,,,,,,
M116,PP,ABC Playground,F,F,F,F
M194,NP,Alfred E. Smith Playground,F,D+,F,F
M226,PP,Augustus St. Gaudens Playground,C,F,C-,C
M165,NP,Baruch Playground,F,F,F,F
M122,PP,Captain Jacob Joseph Playground,F,F,F,F
M171,PP,Cherry Clinton Playground,F,F,F,F
M013,NP,City Hall Park,C+,C+,D,D+
M019,NP,Coleman Playground,F,F,F,F
M242,PP,Collect Pond Park,F,F,F,F


In [208]:
print('The average numeric score is: '+ str(np.around(Score_Filter(NY4Pscores, evparks).loc[:,'tp_palmm':].mean().mean(),2)))

The average numeric score is: 47.78


In [209]:
np.around(Score_Filter(NY4Pscores, evparks).loc[:,'tp_palmm':].mean(),2)

tp_palmm           48.71
tpa_palmm          48.00
tppop_palmm        46.18
tppopdens_palmm    48.24
dtype: float64

In [210]:
average_score(evnewgrades);

The neighborhood average for tp_grade is D
The neighborhood average for tpa_grade is D
The neighborhood average for tppop_grade is D
The neighborhood average for tppopdens_grade is D

The Neighborhood Average is: D


## Bushwick

In [212]:
bushwicknewgrades.sort_values('SIGNNAME')

,TYPECATEGO,SIGNNAME,tp_grade,tpa_grade,tppop_grade,tppopdens_grade
GISPROPNUM,,,,,,
B006,PP,Beattie Square,F,F,F,F
B258,NP,Brevoort Playground,F,D,F,F
B310,NP,Bushwick Playground,F,F,F,F
B219,NP,Callahan-Kelly Playground,F,F,F,F
B302,NP,Charlie's Place,F,F,F,F
Q302,NP,Evergreen Park,F,F,F,F
B402,PP,George Walker Jr. Park,F,F,F,F
B132,NP,Gilbert Ramirez Park,F,F,F,F
B395,NP,Green Central Knoll,F,F,F,F


In [221]:
print('The average numeric score is: '+ str(np.around(Score_Filter(NY4Pscores, bushwick_parks).loc[:,'tp_palmm':].mean().mean(),2)))

The average numeric score is: 40.57


In [222]:
np.around(Score_Filter(NY4Pscores, bushwick_parks).loc[:,'tp_palmm':].mean(),2)

tp_palmm           34.67
tpa_palmm          49.08
tppop_palmm        39.04
tppopdens_palmm    39.50
dtype: float64

In [391]:
average_score(bushwicknewgrades);

The neighborhood average for tp_grade is D-
The neighborhood average for tpa_grade is D
The neighborhood average for tppop_grade is D-
The neighborhood average for tppopdens_grade is D-

The Neighborhood Average is: D-


## Bay Street

In [554]:
baystnewgrades.sort_values('SIGNNAME')

,TYPECATEGO,SIGNNAME,tp_grade,tpa_grade,tppop_grade,tppopdens_grade
GISPROPNUM,,,,,,
R086,PP,Davis Playground,F,F,F,F
R025,NP,Mahoney Playground,F,D+,F,F
R022,LP,Silver Lake Park,F,D-,F,D
R123,NP,Skyline Playground,F,F,F,F
R170,NP,Stapleton Esplanade,F,F,F,F
R061,NP,Stapleton Playground,F,F,F,F
R028,NP,Tappen Park,F,C,F,D
R024,PP,Tompkinsville Park,F,F,D+,B-


In [557]:
perms[perms['GISPROPNUM']=='R028']

,community_board,end_date_time,event_agency,event_borough,event_id,event_location,event_name,event_street_side,event_type,police_precinct,...,start_time,end_time,date,dow,event_clean,park_short,BOROUGH,park_clean,GISPROPNUM,TYPECATEGO
736,"1,",2013-09-29 16:00:00,Parks Department,Staten Island,122457,Tappen Park: Tappen Park - Whole Park,Church Service,NaN,Special Event,"120,",...,12:00:00,16:00:00,2013-09-29,6,church service,Tappen Park,R,tappen park r,R028,Historic House Park
3009,"1,",2016-06-15 17:00:00,Parks Department,Staten Island,289097,Tappen Park: Tappen Park - Whole Park,CHRC Father's Day Event,NaN,Special Event,"120,",...,14:00:00,17:00:00,2016-06-15,2,chrc fathers day event,Tappen Park,R,tappen park r,R028,Historic House Park
3887,"1,",2015-08-14 21:00:00,Parks Department,Staten Island,219323,Tappen Park: Tappen Park - Whole Park,Movie Night Sponsored by HealthPlus,NaN,Special Event,"120,",...,19:00:00,21:00:00,2015-08-14,4,movie night sponsored by healthplus,Tappen Park,R,tappen park r,R028,Historic House Park
4852,"1,",2016-09-06 15:00:00,Parks Department,Staten Island,308543,Tappen Park: Tappen Park - Whole Park,Back To School,NaN,Special Event,"120,",...,11:00:00,15:00:00,2016-09-06,1,back to school,Tappen Park,R,tappen park r,R028,Historic House Park
5965,"1,",2016-09-17 19:00:00,Parks Department,Staten Island,252755,Tappen Park: Tappen Park - Whole Park,SIBHFD We are Family Festival,NaN,Special Event,"120,",...,12:00:00,19:00:00,2016-09-17,5,sibhfd we are family festival,Tappen Park,R,tappen park r,R028,Historic House Park
6018,"1,",2016-06-18 17:00:00,Parks Department,Staten Island,273272,Tappen Park: Tappen Park - Whole Park,True 2 Life Cure Violence,NaN,Special Event,"120,",...,12:00:00,17:00:00,2016-06-18,5,true 2 life cure violence,Tappen Park,R,tappen park r,R028,Historic House Park
9165,"1,",2015-06-21 20:00:00,Parks Department,Staten Island,189127,Tappen Park: Gazebo,Make Music New York,NaN,Special Event,"120,",...,12:00:00,20:00:00,2015-06-21,6,make music new york,Tappen Park,R,tappen park r,R028,Historic House Park
10326,"1,",2014-10-18 15:00:00,Parks Department,Staten Island,144548,Tappen Park: Tappen Park - Whole Park,ITS MY PARK DAY FALL CLEAN UP,NaN,Special Event,"120,",...,12:00:00,15:00:00,2014-10-18,5,its my park day fall clean up,Tappen Park,R,tappen park r,R028,Historic House Park
16106,"1,",2014-06-22 16:00:00,Parks Department,Staten Island,139102,Tappen Park: Tappen Park - Whole Park,NEW GENERATION,NaN,Special Event,"120,",...,11:00:00,16:00:00,2014-06-22,6,new generation,Tappen Park,R,tappen park r,R028,Historic House Park
17664,"1,",2014-08-23 16:00:00,Parks Department,Staten Island,144416,Tappen Park: Tappen Park - Whole Park,SOAP BOX DERBY,NaN,Special Event,"120,",...,11:00:00,16:00:00,2014-08-23,5,soap box derby,Tappen Park,R,tappen park r,R028,Historic House Park


In [223]:
print('The average numeric score is: '+ str(np.around(Score_Filter(NY4Pscores, bayst_parks).loc[:,'tp_palmm':].mean().mean(),2)))

The average numeric score is: 44.59


In [224]:
np.around(Score_Filter(NY4Pscores, bayst_parks).loc[:,'tp_palmm':].mean(),2)

tp_palmm           37.00
tpa_palmm          45.25
tppop_palmm        44.00
tppopdens_palmm    52.12
dtype: float64

In [553]:
average_score(baystnewgrades);

The neighborhood average for tp_grade is F
The neighborhood average for tpa_grade is D-
The neighborhood average for tppop_grade is F
The neighborhood average for tppopdens_grade is D

The Neighborhood Average is: D


## LIC

In [567]:
licnewgrades.sort_values('SIGNNAME')

,TYPECATEGO,SIGNNAME,tp_grade,tpa_grade,tppop_grade,tppopdens_grade
GISPROPNUM,,,,,,
Q045,PP,Andrews Grove,F,F,F,F
Q014,NP,Astoria Heights Playground,F,C-,F,F
Q436,PP,Athens Square,C,F,C+,B
Q267,PP,Court Square Park,F,F,B,B-
Q218,NP,Dutch Kills Playground,F,F,F,F
Q018,PP,Gordon Triangle,F,F,F,F
B043,PP,Greenpoint Playground,F,F,D,F
Q226,NP,Hallets Cove Playground,F,F,F,F
Q471,LP,Hunter's Point South Park,F,B-,C-,F


In [225]:
print('The average numeric score is: '+ str(np.around(Score_Filter(NY4Pscores, lic_parks).loc[:,'tp_palmm':].mean().mean(),2)))

The average numeric score is: 47.58


In [226]:
np.around(Score_Filter(NY4Pscores, lic_parks).loc[:,'tp_palmm':].mean(),2)

tp_palmm           39.95
tpa_palmm          48.00
tppop_palmm        52.10
tppopdens_palmm    50.29
dtype: float64

In [570]:
average_score(licnewgrades);

The neighborhood average for tp_grade is F
The neighborhood average for tpa_grade is D
The neighborhood average for tppop_grade is D
The neighborhood average for tppopdens_grade is D-

The Neighborhood Average is: D


In [582]:
perms[perms['GISPROPNUM']=='Q471']['event_name'].value_counts();

## Park Slope

In [586]:
parkslopenewgrades

,TYPECATEGO,SIGNNAME,tp_grade,tpa_grade,tppop_grade,tppopdens_grade
GISPROPNUM,,,,,,
B126,LP,Red Hook Recreation Area,F,C,D-,F
B232,NP,Dean Playground,F,C,F,F
B326,PP,Cobble Hill Park,C-,F,C+,B-
B019,NP,Carroll Park,C,A-,D+,D+
B113B,NP,Korean War Veterans Plaza,F,D-,F,F
B113C,NP,Columbus Park,C-,C+,D-,D
B210H,NP,Harold Ickes Playground,F,F,F,F
B145,NP,Thomas Greene Playground,F,C-,F,F
B223A,NP,Dimattina Playground,F,D+,F,F


In [227]:
print('The average numeric score is: '+ str(np.around(Score_Filter(NY4Pscores, parkslope_parks).loc[:,'tp_palmm':].mean().mean(),2)))

The average numeric score is: 51.83


In [228]:
np.around(Score_Filter(NY4Pscores, parkslope_parks).loc[:,'tp_palmm':].mean(),2)

tp_palmm           47.44
tpa_palmm          54.88
tppop_palmm        51.64
tppopdens_palmm    53.36
dtype: float64

In [585]:
average_score(parkslopenewgrades);

The neighborhood average for tp_grade is D
The neighborhood average for tpa_grade is D+
The neighborhood average for tppop_grade is D
The neighborhood average for tppopdens_grade is D

The Neighborhood Average is: D


## UWS

In [589]:
uwsnewgrades

,TYPECATEGO,SIGNNAME,tp_grade,tpa_grade,tppop_grade,tppopdens_grade
GISPROPNUM,,,,,,
M022,NP,De Witt Clinton Park,F,D,F,F
M184,PP,Samuel N. Bennerson 2nd Playground,F,F,F,F
M200,NP,Booker T. Washington Playground,F,F,F,F
M219,PP,Tecumseh Playground,F,F,F,F
M220,NP,Frederick Douglass Playground,F,F,F,F
M221,PP,Gutenberg Playground,F,F,F,F
M233,PP,Matthew P. Sapolin Playground,F,F,F,F
M238,PP,Bloomingdale Playground,B+,F,B,B+
M244,PP,Sol Bloom Playground,D,F,D,D


In [229]:
print('The average numeric score is: '+ str(np.around(Score_Filter(NY4Pscores, uws_parks).loc[:,'tp_palmm':].mean().mean(),2)))

The average numeric score is: 51.38


In [230]:
np.around(Score_Filter(NY4Pscores, uws_parks).loc[:,'tp_palmm':].mean(),2)

tp_palmm           52.94
tpa_palmm          53.44
tppop_palmm        48.25
tppopdens_palmm    50.88
dtype: float64

In [588]:
average_score(uwsnewgrades);

The neighborhood average for tp_grade is D+
The neighborhood average for tpa_grade is D+
The neighborhood average for tppop_grade is D
The neighborhood average for tppopdens_grade is D

The Neighborhood Average is: D+
